In [6]:
#!pip install neo4j
import neo4j
from neo4j import GraphDatabase
import pandas as pd

### Nos conectamos a la base de datos

In [7]:
URI = "neo4j+s://06ae1fa1.databases.neo4j.io"
AUTH = ("neo4j","QjwVk3kN-OI5bTt-fg6LZy-F4LMHCpL9HFxuvSuq-OE")

driver = GraphDatabase.driver(URI, auth=AUTH)
with driver.session() as session:
    try:
        session.run("RETURN 1")
        print("Connection to Neo4j established successfully!")
    except Exception as e:
        print(f"Failed to connect to Neo4j: {e}")

Connection to Neo4j established successfully!


### Cargamos los datos

In [16]:
df = pd.read_csv('data_airport/out.opsahl-usairport', sep=' ', skiprows=2, names=['id_from', 'id_to', 'weight'])
df = df.astype({'id_from': int, 'id_to': int, 'weight': int})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28236 entries, 0 to 28235
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   id_from  28236 non-null  int32
 1   id_to    28236 non-null  int32
 2   weight   28236 non-null  int32
dtypes: int32(3)
memory usage: 331.0 KB


In [22]:
unique_ids = pd.DataFrame(pd.concat([df['id_from'], df['id_to']]).unique(), columns=['id'])

In [19]:
# Definir la función para cargar los nodos en Neo4j
def load_nodes(session, nodes):
    query = """
    UNWIND $nodes AS row
    MERGE (n:Node {id: row.id})
    """
    session.run(query, nodes=nodes)

# Definir la función para cargar las relaciones en Neo4j
def load_relationships(session, relationships):
    query = """
    UNWIND $edges AS row
    MATCH (n1:Node {id: row.id_from})
    MATCH (n2:Node {id: row.id_to})
    MERGE (n1)-[r:RELATED {weight: row.weight}]->(n2)
    """
    session.run(query, edges=relationships)


In [23]:
with driver.session() as session:
    # Cargar nodos en Neo4j
    load_nodes(session, unique_ids.to_dict('records'))
    
    # Cargar relaciones en Neo4j
    load_relationships(session, df.to_dict('records'))

In [24]:
driver.close()